# create pipelines and aggregate function

In [8]:
import pymongo
from datetime import datetime
import json

# insert data 

In [3]:
client=pymongo.MongoClient('localhost',27017)


In [ ]:
# load data to database
db=client['library']
collection=db['authors']

with open('data/authors.json','r') as f:
    file=json.load(f)
collection.insert_many(file)



InsertManyResult([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], acknowledged=True)

In [14]:
collection=db['users']

with open('data/users.json','r') as f:
    file=json.load(f)
collection.insert_many(file)

collection=db['books']

with open('data/books.json','r') as f:
    file=json.load(f)
collection.insert_many(file)

InsertManyResult([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], acknowledged=True)

# pipeline
[
    
    {}, stage 1 source
    
    {}, stage 2 
]

In [32]:
# how many uses are active
collection=db['users']
pipeline=[
    {'$match':{'gender':'female'}},
    {'$group':{'_id':'$isActive','count isActive':{'$sum':1}}}
]

result=collection.aggregate(pipeline)
for data in result:
    print(data)

{'_id': False, 'count isActive': 50}


In [ ]:
#what is the average age of all users/gender
collection=db['users']
pipeline=[
    {'$group':{'_id':'$gender','averageAge':{'$avg':'$age'}}}

]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'female', 'averageAge': 70.0}
{'_id': 'male', 'averageAge': 71.0}


In [ ]:
# list the top 2 common favorute fruits among users
pipeline=[
    {'$group':{'_id':'$favoriteFruit','count':{'$sum':1}}},
    {'$sort':{'count':-1}},
    {'$limit':2}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'banana', 'count': 20}
{'_id': 'pineapple', 'count': 20}


In [57]:
# find the total number of males and females 
pipeline=[
    {'$group':{'_id':'$gender','count':{'$sum':1}}}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'female', 'count': 50}
{'_id': 'male', 'count': 50}


In [ ]:
# which country has the highest number of registered user 


{'gender': 1}


In [ ]:
pipeline=[
    {'$group':{'_id':'$company.location.country','user_count':{'$sum':1}}},
    {'$sort':{'user_count':-1}},
    {'$limit':1}
]
result=collection.aggregate(pipeline)
for r in result:
    print(r)

{'_id': 'USA', 'user_count': 100}
